# About: Hivemall - Ready! on Hive

----

すでにHiveがインストールされた環境にHivemallをインストールします。

## *Operation Note*

*This is a cell for your own recording. ここに経緯を記述*

# 操作対象クラスタの設定

起動したい対象のクラスタ名を設定する。

In [1]:
target_group = 'hadoop_all_testcluster'

対象クラスタにAnsibleでpingできることを確認する。

In [2]:
!ansible -m ping {target_group}

XXX.XXX.XXX.70 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.73 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.112 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.72 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.71 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.113 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.114 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


インストール対象は、 `hadoop_client` というグループに所属しているものとする。

In [12]:
!ansible -m ping hadoop_hive -l {target_group}

XXX.XXX.XXX.72 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


# 設定の定義

インストール対象パスなど、設定を定義する。

In [4]:
hivemall_install_path = '/usr/local/lib'

インストールするhivemallのバージョンを定義する。

https://github.com/myui/hivemall

In [5]:
hivemall_version = 'v0.4.0-2'

Playbookなど定義するための一時ディレクトリを定義しておく。

In [6]:
import os
import tempfile

work_dir = tempfile.mkdtemp()
work_dir

'/tmp/tmpmitk4u'

# hivemallのインストール

hivemallのjarファイルはGitからcloneしてくる・・・まずは対象ホストにgitをインストールする。

In [7]:
!ansible -b -m yum -a 'name=git state=latest' hadoop_hive -l {target_group}

XXX.XXX.XXX.72 | SUCCESS => {
    "changed": false, 
    "msg": "", 
    "rc": 0, 
    "results": [
        "All packages providing git are up to date", 
        ""
    ]
}


リポジトリからcloneしてくる。

In [9]:
!ansible -b -m git -a 'repo=https://github.com/myui/hivemall.git dest={ hivemall_install_path }/hivemall-{ hivemall_version } version={ hivemall_version }' \
         hadoop_hive -l {target_group}

XXX.XXX.XXX.72 | SUCCESS => {
    "after": "03ad62db71db4617bd65499eb4c657ade651b68a", 
    "before": null, 
    "changed": true
}


`hivemall-with-dependencies.jar` が作成されていることを確認する。

In [10]:
!ansible -a 'ls -la { hivemall_install_path }/hivemall-{ hivemall_version }/target/hivemall-with-dependencies.jar' \
         hadoop_hive -l {target_group}

XXX.XXX.XXX.72 | SUCCESS | rc=0 >>
-rw-r--r-- 1 root root 1616518 Aug 23 12:43 /usr/local/lib/hivemall-v0.4.0-2/target/hivemall-with-dependencies.jar



`.hiverc` を設定する。

In [11]:
with open(os.path.join(work_dir, '.hiverc'), 'w') as f:
    f.write('''add jar {hivemall_install_path}/hivemall-{hivemall_version}/target/hivemall-with-dependencies.jar;
source {hivemall_install_path}/hivemall-{hivemall_version}/scripts/ddl/define-all.hive;'''.format(hivemall_install_path=hivemall_install_path,
                                                                                                  hivemall_version=hivemall_version))
!cat {work_dir}/.hiverc

add jar /usr/local/lib/hivemall-v0.4.0-2/target/hivemall-with-dependencies.jar;
source /usr/local/lib/hivemall-v0.4.0-2/scripts/ddl/define-all.hive;

ユーザに .hiverc を配布する。ここでは、対象マシンの全ユーザに登録してみる。

In [16]:
!ansible -m fetch -a 'src=/etc/passwd dest={work_dir}/passwd flat=yes' hadoop_hive -l {target_group}

XXX.XXX.XXX.72 | SUCCESS => {
    "changed": true, 
    "checksum": "4efbb79064d7976369104d548c16320b9e0209cf", 
    "dest": "/tmp/tmpmitk4u/passwd", 
    "md5sum": "a5ff11d699f902e50805382376fd89f1", 
    "remote_checksum": "4efbb79064d7976369104d548c16320b9e0209cf", 
    "remote_md5sum": null
}


ここでは、shellがbashかつHDPインストール時に登録されたユーザでなければ登録対象とする。

In [21]:
home = {}
hdp_sys_user = ['hdfs', 'mapred', 'zookeeper', 'yarn', 'spark', 'hive', 'hbase']
with open(os.path.join(work_dir, 'passwd'), 'r') as file_object:
    for line in file_object:
        fields = line.strip().split(":")
        if fields[-1] == '/bin/bash' and fields[0] not in hdp_sys_user:
            home[fields[0]] = fields[-2]

home.items()

[('ansible', '/home/ansible'), ('root', '/root'), ('yazawa', '/home/yazawa')]

各ユーザのhomeに.hivercを配布する。

In [22]:
for owner, path in home.items():
    !ansible -b -m copy -a 'src={work_dir}/.hiverc dest={path}/.hiverc owner={owner} group={owner}' \
             hadoop_hive -l {target_group}

XXX.XXX.XXX.72 | SUCCESS => {
    "changed": true, 
    "checksum": "4fe3e66eda923ca04cf13e20ea6c9e68b44f1443", 
    "dest": "/home/ansible/.hiverc", 
    "gid": 500, 
    "group": "ansible", 
    "md5sum": "051b211b9c79d4243d0a3a9e38052b8d", 
    "mode": "0644", 
    "owner": "ansible", 
    "size": 148, 
    "src": "/home/ansible/.ansible/tmp/ansible-tmp-1471925391.0-90284739064595/source", 
    "state": "file", 
    "uid": 500
}
XXX.XXX.XXX.72 | SUCCESS => {
    "changed": true, 
    "checksum": "4fe3e66eda923ca04cf13e20ea6c9e68b44f1443", 
    "dest": "/root/.hiverc", 
    "gid": 0, 
    "group": "root", 
    "md5sum": "051b211b9c79d4243d0a3a9e38052b8d", 
    "mode": "0644", 
    "owner": "root", 
    "size": 148, 
    "src": "/home/ansible/.ansible/tmp/ansible-tmp-1471925393.17-73355278997319/source", 
    "state": "file", 
    "uid": 0
}
XXX.XXX.XXX.72 | SUCCESS => {
    "changed": true, 
    "checksum": "4fe3e66eda923ca04cf13e20ea6c9e68b44f1443", 
    "dest": "/home/yazawa/.hiver

# 動作確認

Hivemallのバージョンを確認する。

In [25]:
!ansible -m shell -a 'echo "SELECT HIVEMALL_VERSION();" | hive' hadoop_hive -l {target_group}

XXX.XXX.XXX.72 | SUCCESS | rc=0 >>
hive> SELECT HIVEMALL_VERSION();
0.4.0
hive> WARNING: Use "yarn jar" to launch YARN applications.

Logging initialized using configuration in file:/etc/hive/conf/hive-log4j.properties
OK
Time taken: 1.052 seconds, Fetched: 1 row(s)



# 後始末

一時ディレクトリを削除する。

In [23]:
!rm -fr {work_dir}